In [1]:
from torchvision import transforms
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import timeit
import pandas as pd
import gc
from pathlib import Path

import utils.image as im
import utils.cam as cam
import utils.grabcut as gcut
import utils.voc as voc
import utils.json as json
import utils.path as path
from utils.VOCSegmentation import VOCSegmentation

C:\Users\Nicol\miniconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Nicol\miniconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
FB_threshold = 0.5

In [3]:
root_path = path.goback_from_current_dir(0)

json_path = root_path + 'json\\'

output_path  = root_path + r'sessions\2 Object grabcut-cam\threshold_0' + str(int(FB_threshold * 10)) + '\\'

In [4]:
Path(output_path).mkdir(parents=True, exist_ok=True)

In [5]:
camnet = cam.Cam()

In [6]:
data_tbl = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Object')

In [7]:
data = iter(torch.utils.data.DataLoader(data_tbl,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 2))

In [8]:
annotations = json.open_json(json_path + "voc-object-annotations-clean")
N = len(annotations)
annotations = iter(annotations.items())

In [9]:
IoU  = []
Acc  = []
time = []

for i in range(N):
    start = timeit.default_timer()
    gc.collect()
    
    img, sgm = next(data)
    img, sgm = torch.squeeze(img), torch.squeeze(sgm)

    img_pil = img
    img_cv2 = im.pil_to_cv2(img.numpy())
    sgm     = im.f1_to_f255(sgm.numpy())
    
    name, annots = next(annotations)
    
    for j, annot in enumerate(annots):
        c, d, cbbox = annot
        
        img_cam = camnet.get_top(img_pil, c)
        img_cam = cam.cam_process(img_pil, img_cam)
        img_cam = im.bitwise_and(img_cam, im.cbbox_mask(img_cv2, cbbox))
        gcmask_cam = cam.cam_to_gcmask(img_cam, a = 0.0, b = FB_threshold, c = 1.0)
        _, _, pred = gcut.grabcut(img_cv2, gcmask_cam, mode = 'MASK')
        pred    = im.bitwise_and(pred, im.cbbox_mask(img_cv2, cbbox)).astype(bool)
        
        true = voc.true_mask(sgm, j+1).astype(bool)
        
        IoU.append(voc.IoU(true, pred, sgm))
        Acc.append(voc.accuracy(true, pred, sgm))
        
    stop = timeit.default_timer()
    time.append(stop - start)
    if i % 20 == 0:
        print(f'Image nb {i}')
        print(f'Time spent               = ' + voc.time_str(np.sum(time)))
        print(f'Estimated time remaining = ' + voc.time_str(np.mean(time) * (N - 1 - i)))
        print(f'Mean IoU                 =  {np.mean(IoU) :.3f}')
        print(f'Mean Acc                 =  {np.mean(Acc) :.3f}')
        print()

Image nb 0
Time spent               =   0m00s
Estimated time remaining =  35m37s
Mean IoU                 =  0.135
Mean Acc                 =  0.978

Image nb 20
Time spent               =   0m13s
Estimated time remaining =  30m28s
Mean IoU                 =  0.234
Mean Acc                 =  0.886

Image nb 40
Time spent               =   0m27s
Estimated time remaining =  31m52s
Mean IoU                 =  0.228
Mean Acc                 =  0.906

Image nb 60
Time spent               =   0m42s
Estimated time remaining =  33m04s
Mean IoU                 =  0.227
Mean Acc                 =  0.919

Image nb 80
Time spent               =   0m57s
Estimated time remaining =  33m22s
Mean IoU                 =  0.270
Mean Acc                 =  0.932

Image nb 100
Time spent               =   1m12s
Estimated time remaining =  33m29s
Mean IoU                 =  0.259
Mean Acc                 =  0.932

Image nb 120
Time spent               =   1m29s
Estimated time remaining =  34m25s
Mean IoU   

Image nb 1080
Time spent               =  11m38s
Estimated time remaining =  19m43s
Mean IoU                 =  0.302
Mean Acc                 =  0.929

Image nb 1100
Time spent               =  11m51s
Estimated time remaining =  19m30s
Mean IoU                 =  0.302
Mean Acc                 =  0.929

Image nb 1120
Time spent               =  12m02s
Estimated time remaining =  19m15s
Mean IoU                 =  0.303
Mean Acc                 =  0.929

Image nb 1140
Time spent               =  12m13s
Estimated time remaining =  18m59s
Mean IoU                 =  0.303
Mean Acc                 =  0.928

Image nb 1160
Time spent               =  12m28s
Estimated time remaining =  18m49s
Mean IoU                 =  0.304
Mean Acc                 =  0.928

Image nb 1180
Time spent               =  12m39s
Estimated time remaining =  18m33s
Mean IoU                 =  0.303
Mean Acc                 =  0.928

Image nb 1200
Time spent               =  12m50s
Estimated time remaining =  18m18

Image nb 2160
Time spent               =  22m42s
Estimated time remaining =   7m54s
Mean IoU                 =  0.323
Mean Acc                 =  0.926

Image nb 2180
Time spent               =  22m56s
Estimated time remaining =   7m41s
Mean IoU                 =  0.323
Mean Acc                 =  0.926

Image nb 2200
Time spent               =  23m12s
Estimated time remaining =   7m30s
Mean IoU                 =  0.322
Mean Acc                 =  0.926

Image nb 2220
Time spent               =  23m25s
Estimated time remaining =   7m17s
Mean IoU                 =  0.322
Mean Acc                 =  0.926

Image nb 2240
Time spent               =  23m40s
Estimated time remaining =   7m05s
Mean IoU                 =  0.322
Mean Acc                 =  0.926

Image nb 2260
Time spent               =  23m54s
Estimated time remaining =   6m53s
Mean IoU                 =  0.321
Mean Acc                 =  0.926

Image nb 2280
Time spent               =  24m07s
Estimated time remaining =   6m40

In [10]:
print(f'Mean IoU = {np.mean(IoU):.3f}')
print(f'Mean Acc = {np.mean(Acc):.3f}')

Mean IoU = 0.310
Mean Acc = 0.924


In [11]:
df = pd.read_csv(json_path + 'voc-object-annotations-clean.csv')

In [12]:
df['IoU'] = IoU
df['Accuracy'] = Acc

In [13]:
df.to_csv(output_path + 'IoU.csv', index = False)